In [171]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [172]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
955,tt0280760,9000000,4777465,Igby Goes Down,Kieran Culkin|Claire Danes|Jeff Goldblum|Jared...,Burr Steers,Insanity is relative.,A young man's peculiar upbringing renders him ...,97,Comedy|Drama,United Artists,9/13/2002,6.5,2002
1642,tt0360201,31000000,41512007,Wimbledon,Kirsten Dunst|Paul Bettany|Sam Neill|James McA...,Richard Loncraine,She's the golden girl. He's the longshot. It's...,A pro tennis player has lost his ambition and ...,98,Comedy|Romance,Universal Pictures,9/13/2004,5.7,2004
1793,tt0770772,11000000,13196245,I Think I Love My Wife,Chris Rock|Kerry Washington|Gina Torres|Steve ...,Chris Rock,In marriage no one can hear you scream.,Richard Cooper (Rock) is a married man and fat...,90,Comedy|Romance,Fox Searchlight Pictures,3/16/2007,5.3,2007
841,tt1596346,18000000,47088990,Soul Surfer,AnnaSophia Robb|Helen Hunt|Lorraine Nicholson|...,Sean McNamara,"When you come back from a loss, beat the odds,...",Soul Surfer is the inspiring true story of tee...,106,Action|Drama,Mandalay Pictures|TriStar Pictures|Brookwell-M...,4/8/2011,6.9,2011
1284,tt1747958,25500000,2415472,Blood Ties,Clive Owen|Billy Crudup|Marion Cotillard|Mila ...,Guillaume Canet,Crime runs in the family.,"Two brothers, on either side of the law, face ...",128,Thriller|Crime|Drama,Wild Bunch|LGM Productions|Les Productions du ...,8/22/2013,5.8,2013


In [173]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


In [174]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


# Предобработка

In [175]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...
data['release_date'] = pd.to_datetime(data['release_date'])
data['profit'] = data['revenue'] - data['budget']

def frequent_genre(df):
    c = Counter()
    for i in range(len(df)):
        for j in df.iloc[i].genres.split('|'):
            c[j] += 1
    return c.most_common(1)

def profit_genre(df):
    c = Counter()
    for i in range(len(df)):
        for y in df.iloc[i].genres.split('|'):
            c[y] += 1
    return c.most_common(1)

def director_revenue(df):
    c = Counter()
    for i in range(len(df)):
        for j in df.loc[i].director.split('|'):
            c[j] += df.loc[i].revenue
    return c.most_common(1)

def director_action(df):
    c = Counter()
    for i in range(len(df)):
        for j in df.iloc[i].director.split('|'):
            if 'Action' in df.iloc[i].genres:
                c[j] += 1
    return c.most_common(1)

def actors_revenue_2012(df):
    c = Counter()
    for i in range(len(df)):
        for j in df.iloc[i].cast.split('|'):
            c[j] += df.iloc[i].revenue
    return c.most_common(1)

def high_value_actor(df):
    c = Counter()
    for i in range(len(df)):
        for j in df.iloc[i].cast.split('|'):
            c[j] += 1
    return c.most_common(1)

def nc_genre(df):
    c = Counter()
    for i in range(len(data)):
        for j in data.iloc[i].genres.split('|'):
            if 'Nicolas Cage' in data.iloc[i].cast:
                c[j] +=1
    return c.most_common(1)

def profit_year(df):
    c = Counter()
    for x in df.release_year.unique():
        c[x] = data[data.release_year == x].revenue.sum()
    return c.most_common(1)

def longest_title_studio(df):
    lenth = Counter()
    amount = Counter()
    for x in range(len(df)):
        for y in df.iloc[x].production_companies.split('|'):
            lenth[y] += len(data.iloc[x].original_title)
            amount[y] +=1
    for x in lenth.keys():
        lenth[x] = lenth[x] / amount[x]
    return lenth.most_common(1)

def more_words_studio(df):
    lenth = Counter()
    amount = Counter()
    for x in range(len(data)):
        for y in df.iloc[x].production_companies.split('|'):
            lenth[y] += len(df.iloc[x].overview)
            amount[y] +=1
    for x in lenth.keys():
        lenth[x] = lenth[x] / amount[x]
    return lenth.most_common(1)

def best_percent_movies(df):
    best_movies = []
    for x in range(int(len(df) * 0.01 //1)):
        best_movies.append(data.sort_values(by='vote_average', ascending = False).original_title.iloc[x])
    return best_movies

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [176]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides'
# если ответили верно, можете добавить комментарий со значком "+"

In [177]:
# тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

ВАРИАНТ 2

In [178]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [179]:
# думаю логику работы с этим словарем вы уже поняли, 
# по этому не буду больше его дублировать
answers['2'] = 'Gods and Generals'

In [180]:
data[data.runtime == data.runtime.max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

# 3. Какой из фильмов самый короткий (в минутах)?





In [181]:
data[data['runtime'] == data['runtime'].min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

In [182]:
answers['3'] = 'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [183]:
data.runtime.mean()

109.6585494970884

In [184]:
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [185]:
data.runtime.median()

107.0

In [186]:
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [187]:
data[data['profit'] == data['profit'].max()].original_title

239    Avatar
Name: original_title, dtype: object

In [188]:
answers['6'] = 'Avatar'

# 7. Какой фильм самый убыточный? 

In [189]:
data[data['profit'] == data['profit'].min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

In [190]:
answers['7'] = 'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [191]:
len(data[data.revenue > data.budget])

1478

In [192]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [193]:
data[(data.release_year == 2008)].sort_values(by='revenue', ascending = False).original_title.iloc[0]

'The Dark Knight'

In [194]:
answers['9'] = 'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [195]:
data[(data.release_year >= 2012) & (data.release_year <= 2014)].sort_values(by='profit').original_title.iloc[0]

'The Lone Ranger'

In [196]:
answers['10'] = 'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [197]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале
display(frequent_genre(data))

[('Drama', 782)]

In [198]:
answers['11'] = 'Drama'

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [199]:
profit_genre(data)

[('Drama', 782)]

In [200]:
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [201]:
director_revenue(data)

[('Peter Jackson', 6490593685)]

In [202]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [203]:
director_action(data)

[('Robert Rodriguez', 9)]

In [204]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [205]:
actors_revenue_2012(data[data.release_year == 2012])

[('Chris Hemsworth', 2027450773)]

In [206]:
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [207]:
high_value_actor(data[data.budget > data.budget.mean()])

[('Matt Damon', 18)]

In [208]:
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [209]:
nc_genre(data)

[('Action', 17)]

In [210]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [211]:
data[data.production_companies.str.contains('Paramount Pictures')].sort_values(by='profit').original_title.iloc[0]

'K-19: The Widowmaker'

In [212]:
answers['18'] = 'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [213]:
profit_year(data)

[(2015, 25449202382)]

In [214]:
answers['19'] = '2015'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [215]:
c = Counter()
for x in data.release_year.unique():
    c[x] = data[(data.release_year == x) & (data.production_companies.str.contains('Warner Bros'))].revenue.sum()
c.most_common(1)

[(2014, 3243064519)]

In [216]:
answers['20'] = '2014'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [217]:
months = Counter()
for x in range(len(data)):
    months[data.iloc[x].release_date.month] +=1
c.most_common(1)

[(2014, 3243064519)]

In [218]:
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [219]:
months[6]+months[7]+months[8]

450

In [220]:
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [221]:
c = Counter()
for x in range(len(data)):
    if data.iloc[x].release_date.month == 12 or data.iloc[x].release_date.month == 1 or data.iloc[x].release_date.month == 2:
        c[data.iloc[x].director] += 1
c.most_common(1)

[('Peter Jackson', 7)]

In [222]:
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [223]:
longest_title_studio(data)

[('Four By Two Productions', 83.0)]

In [224]:
answers['24'] = 'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [225]:
more_words_studio(data)

[('Midnight Picture Show', 1000.0)]

In [226]:
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [227]:
best_percent_movies(data)

['The Dark Knight',
 'Interstellar',
 'The Imitation Game',
 'Inside Out',
 'Room',
 'The Wolf of Wall Street',
 'Gone Girl',
 '12 Years a Slave',
 'Guardians of the Galaxy',
 'The Lord of the Rings: The Return of the King',
 'Memento',
 'Inception',
 'The Pianist',
 'The Grand Budapest Hotel',
 'Her',
 'Spotlight',
 'Big Hero 6',
 'The Fault in Our Stars']

In [228]:
answers['26'] = ['Inside Out', 'The Dark Knight', '12 Years a Slave']

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [229]:
from itertools import combinations
actor_list = data.cast.str.split('|').tolist()
combo_list=[]
for i in actor_list:
    for j in combinations(i, 2):
        combo_list.append(' '.join(j))
combo_list = pd.DataFrame(combo_list)
combo_list.columns = ['actor_combinations']
combo_list.actor_combinations.value_counts().head(3)

Daniel Radcliffe Emma Watson     8
Daniel Radcliffe Rupert Grint    8
Rupert Grint Emma Watson         7
Name: actor_combinations, dtype: int64

In [230]:
answers['27'] = 'Daniel Radcliffe Emma Watson'

# Submission

In [231]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 110,
 '5': 107,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': '2015',
 '20': '2014',
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
 '27': 'Daniel Radcliffe Emma Watson'}

In [232]:
# и убедиться что ни чего не пропустил)
len(answers)

27